In [2]:
import re
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

year = 2015
df = pd.read_csv('./gaon_final_data.csv', index_col=0)
df = df.loc[df['year']==year]

# 노래제목 리스트
music_list = df['music'].unique().tolist()
music_list = music_list[500:]

# singer
for music in music_list:
    try:
        singer = df['singer'].loc[df['music']==music].tolist()[0]
    except:
        print('오류', music)


# 노래 id 딕셔너리 만들기 (key:노래id값, value:노래 제목)
id_dict={}
for music in music_list:
    singer = df['singer'].loc[df['music']==music].tolist()[0]

    # 가수이름 - 정규표현식으로 괄호 안의 내용 모두 삭제
    if singer.find('(') != -1:
        singer = re.sub('\([^)]*\)','', singer) 
    else:
        pass

    # 노래제목 - 정규표현식으로 괄호 안의 내용 모두 삭제
    if music.find('(') != -1:
        music = re.sub('\([^)]*\)','', music) 
    else:
        pass
    
    singer = singer.rstrip()
    music = music.rstrip()

    # 공백을 '+'로 바꿔줌
    music = music.replace(' ', '+')
    singer = singer.replace(' ', '+')

    # 특수문자 제거
    singer = re.sub('[-=+#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', "",singer)

    print(f'music:{music}, singer:{singer}')

    try:
        URL = f'https://www.melon.com/search/total/index.htm?q={music}+{singer}'
        header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
        response = requests.get(URL,headers=header)
        time.sleep(1)
        soup = BeautifulSoup(response.text, 'html.parser')

        # try:
        links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
        #frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input

        music_id = links.attrs['value']
        print(music_id)
    except:
        print(f'오류, music:{music}')
        pass

    # +을 공백로 바꿔줌
    music = music.replace('+', ' ')
    singer = singer.replace('+', ' ')

    id_dict[music_id] = [music, singer]
    time.sleep(1)

# 노래 id만 있는 리스트 
id_list = list(id_dict.keys())



# 데이터 뽑아오기
result = []

for id in id_list:
    music = id_dict[id][0]
    singer = id_dict[id][1]
    print(f'현재 {music} 크롤링 중')

    try:
        URL_1 = f'https://www.melon.com/song/detail.htm?songId={id}'
        response_1 = requests.get(URL_1,headers=header)
        time.sleep(1)
        soup = BeautifulSoup(response_1.text, 'html.parser')

        # 발매일, 장르
        release_date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
        category = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
        

        # 좋아요 개수 json 접근
        URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={id}'
        response_2 = requests.get(URL_2,headers=header).text
        time.sleep(1)
        like_json = json.loads(response_2)
        music_like = like_json['contsLike'][0]['SUMMCNT']

        # 코멘트 개수 json 접근
        URL_3 = f'https://www.melon.com/song/songReviewCnt.json?songId={id}'
        response_3 = requests.get(URL_3,headers=header).text
        time.sleep(1)

        comment_json = json.loads(response_3)
        comment_count = comment_json['reviewValidCmtCnt']

        temp = {
        'music':music,
        'singer':singer,
        'music_id':id,
        'release_date':release_date,
        'category':category,
        'music_like':music_like,
        'comment_count':comment_count,
    }

        result.append(temp)
    except:
        print(f'오류, music:{music}')
        pass


df = pd.DataFrame(result)
df.to_csv(f'./melon_data_{year}_2.csv')



music:Thank+U, singer:케이윌
5697154
music:Excuse+Me, singer:베스티
5688724
music:너도+나처럼, singer:화요비
오류, music:너도+나처럼
music:흔한+이별, singer:송유빈,김나영
5695067
music:All+Day, singer:천재노창
오류, music:All+Day
music:낯선,+새벽, singer:에피톤프로젝트
5686661
music:Double+Jack, singer:보아
5692642
music:알아+알아, singer:데이비드오
5690886
music:My+Valentine, singer:정엽
5698860
music:맴돌아, singer:포스트맨
5687750
music:Love+Affair, singer:45RPM
5697137
music:마음, singer:아이유
5700639
music:View, singer:샤이니
5700879
music:꺼내+먹어요, singer:자이언티
5702706
music:Love+Sick, singer:샤이니
5700878
music:이럴거면+그러지말지, singer:백아연
5704397
music:동네술집, singer:허각,정인
5704765
music:Odd+Eye, singer:샤이니
5700877
music:재연, singer:샤이니
5700887
music:너의+노래가+되어, singer:샤이니
5700883
music:몸매, singer:박재범
5708013
music:Romance, singer:샤이니
5700880
music:잘됐으면+좋겠다, singer:홍대광
5697094
music:이별의+길, singer:샤이니
5700882
music:Black+Hole, singer:샤이니
5700886
music:Trigger, singer:샤이니
5700881
music:Woof+Woof, singer:샤이니
5700885
music:To+Be+With+You, singer:김연우
5709752
music:더+가까이, 